# Lab. 2: Stochastic global optimization

## Introduction

#### <u>The goal of this laboratory is to study the application of stochastic global search algorithms on different benchmark functions.</u>

We will study two methods:
- *DIRECT*
- *Basin-hopping*

Moreover, we will study the effect of their parameters on the outcome of the search.

---

Getting started: the following code cell contains the core functions that we will use. Hence, **remember to run it every time the runtime is reconnected**.

You can find a wrapper function for the  two algorithm, together with a wrapper for the benchmark function.
As regards the *OptFun* class, the constructor takes as input a benchmark function (we will see later what functions are available). The relevant methods  are 4:
1.   *Minima*: return the minimum of the function. The position can be obtained by the parameter *position* and the function value from the *score* parameter.
2.   *Bounds*: returns where the function is defined
3.   *Heatmap*: show a heatmap of the function highlighting the points visited by the local search (use with 2d function)
4.   *plot*: show the trend of the points visited by the local search (use with 1d function)
5.   *trend*: show the best points find during the optmization process.

Each instance of *OptFun* stores the history of the point at which the function has been evaluated. The history is never cleaned and can be obtained through *OptFun.history*. Hence, if you reuse the class instance remember to clean the history (*OptFun.history = list()*).

---

The benchmark functions available comes from the *benchmark_functions* library (imported as *bf*).
Example of the functions that can be used are the *Hypersphere*, the *Rastrign* the *DeJong5* and the Keane.
The complete list of functions available can be found at this [link](https://gitlab.com/luca.baronti/python_benchmark_functions) or you can print it with *dir(bf)*.


#### Base code to run every time the runtime is reconnected

In [ ]:
import inspect
from copy import deepcopy
from typing import Optional, Sequence

import benchmark_functions as bf
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import OptimizeResult as spResult
from scipy.optimize import basinhopping as spbasinhopping
from scipydirect import OptimizeResult as spdResult
from scipydirect import minimize as spdirect

%matplotlib inline

plt.rcParams["figure.figsize"] = (22, 8)

In [ ]:
class OptFun:
    def __init__(self, wf: bf.BenchmarkFunction) -> None:
        self.f = wf
        self.history: list[list[float]] = []

    def __call__(self, x0: list[float]) -> float:
        self.history.append(deepcopy(x0))
        return self.f(x0)  # type: ignore

    @property
    def name(self) -> str:
        return self.f.name()

    def minima(self) -> list[bf.fil.Optimum]:
        """
        Returns a list of Optimum objects of the known global minima. If there aren't any minima, an empty list value will be returned instead;

        Returns:
        - List of objects of class "benchmark_functions.functions_info_loader.Optimum"
        - For each object:
          - Access to 'position' parameter to get the axis values
          - Access to 'score' parameter to get the value of the function
        """
        return self.f.minima()

    def bounds(self) -> list[tuple[float, float]]:
        return self._convert_bounds(self.f.suggested_bounds())

    def found_minimum(self) -> list[float]:
        minimum = self.history[0]
        for x in self.history:
            if self.f(x) < self.f(minimum):  # type: ignore
                minimum = x
        return minimum

    def plot(self, fn: Optional[str] = None) -> None:
        plt.clf()
        ax1: plt.Axes
        ax2: plt.Axes

        f, (ax1, ax2) = plt.subplots(1, 2)
        f.suptitle("Benchmark Function: " + self.name)

        # heatmap
        bounds_lower, bounds_upper = self.f.suggested_bounds()
        x = np.linspace(bounds_lower[0], bounds_upper[0], 100)
        if self.f.n_dimensions() > 1:
            y = np.linspace(bounds_lower[1], bounds_upper[1], 100)
            X, Y = np.meshgrid(x, y)
            Z = np.asarray(
                [
                    [self.f((X[i][j], Y[i][j])) for j in range(len(X[i]))]
                    for i in range(len(X))
                ]
            )
        else:
            raise ValueError("Function has only one dimension")

        ax1.contour(x, y, Z, 15, linewidths=0.5, colors="k")  # hight lines
        contour = ax1.contourf(
            x, y, Z, 15, cmap="viridis", vmin=Z.min(), vmax=Z.max()
        )  # heat map
        ax1.set_xlabel("x")
        ax1.set_ylabel("y")
        cbar = plt.colorbar(contour, ax=ax1)
        cbar.set_label("z")
        if len(self.history) > 0:  # plot points
            xdata = [x[0] for x in self.history]
            xdata = xdata[1:]
            ydata = [x[1] for x in self.history]
            ydata = ydata[1:]

            ax1.plot(xdata, ydata, "or-", markersize=5, linewidth=1.5, label="Path")
            ax1.plot(
                xdata[0],
                ydata[0],
                marker="X",
                markersize=13,
                color="black",
                label="Start",
            )

            ax1.plot(
                xdata[-1],
                ydata[-1],
                marker="P",
                markersize=13,
                color="white",
                label="End",
            )
            ax1.legend(loc="upper right")

        # convergence
        values = [self.f(v) for v in self.history]
        values = values[1:]
        min: float = self.f.minima()[0].score  # type: ignore

        ax2.plot(values, label="Scores", marker="o")

        ax2.axhline(min, color="r", label="optimum", linestyle="dashed")
        ax2.set_xlabel("Iterations/Evaluations")  # TODO
        ax2.set_ylabel("f(x)")
        ax2.legend()

        if fn is not None:
            plt.savefig(fn, dpi=400)
        plt.show()

    def _convert_bounds(
        self, bounds: tuple[list[float], list[float]]
    ) -> list[tuple[float, float]]:
        new_bounds = []
        for i in range(len(bounds[0])):
            new_bounds.append((bounds[0][i], bounds[1][i]))
        return new_bounds

    def current_calls(self) -> int:
        return len(self.history)


def grid_search(
    f: OptFun,
    step_size: Optional[float] = None,
    number_of_steps: Optional[float] = None,
    minimization: bool = True,
) -> float:
    """
    Optimizes a function by using the grid_search algorithm.

    - f: function to optimize, an instance of OptFun
    - step_size: the step size
    - number_of_steps: the total number of steps
    - minimization: boolean, True if looking for minimum, False if looking for maximum; defauls True

    Returns:
    - best value found
    """

    if step_size is not None:
        range_step_size = step_size
    elif number_of_steps is not None:
        range_step_size = int(np.floor(np.sqrt(number_of_steps)))
    else:
        raise ValueError("Please provide at least the step_size or the number_of_steps")

    bounds = f.bounds()
    best = float("inf") if minimization else float("sup")

    for x in np.arange(bounds[0][0], bounds[0][1], range_step_size):
        for y in np.arange(bounds[1][0], bounds[1][1], range_step_size):
            current = f([x, y])
            if minimization:
                if current < best:
                    best = current
            else:
                if current > best:
                    best = current

    return best


def direct(
    func: OptFun,
    eps: float = 1e-4,
    maxiter: int = 1000,
    maxfneval: int = 1000,
    optima: Optional[int] = None,
    error_tollerance: Optional[int] = None,
) -> tuple[spdResult, list[float], float]:
    """
    Optimizes a function by using the DIRECT algorithm.

    - f: function to optimize, an instance of OptFun
    - eps: regulates the trade-off between local and global exploration.
            The smaller the eps, the finer the granularity of the search.
    - maxiter: maximum number of iterations
    - maxfneval: maximum number of function evaluations
    """
    bounds = func.bounds()
    results = spdirect(
        func,
        bounds=bounds,
        eps=eps,
        maxT=maxiter,
        maxf=maxfneval,
        fglobal=-1e100,
        fglper=0.001,
        volper=-1.0,
        sigmaper=-1.0,
    )
    # best_position: list[float] = results.x
    # best_score: float = results.fun
    history: list[float] = [func.f(v) for v in func.history]  # type: ignore
    history = history[1:]
    best_score = min(history)
    best_position = func.history[history.index(best_score)]
    return results, best_position, best_score


def basinhopping(
    f: OptFun,
    x0: Sequence[float],
    T: float = 1.0,
    stepsize: float = 0.5,
    stepsize_interval: int = 50,
    maxiter: int = 1000,
    bounds: Optional[Sequence[tuple[float, float]]] = None,
) -> spResult:
    """
    Optimizes a function by using the Basin-hopping algorithm.

    - f: function to optimize, an instance of OptFun
    - x0: starting point for the search process
    - T: temperature parameter
    - stepsize: maximum step size for random displacement
    - stepsize_interval: interval for the update of the step size
    - maxiter: maximum number of iterations
    """
    minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
    return spbasinhopping(
        f,
        x0,
        niter=maxiter,
        T=T,
        stepsize=stepsize,
        minimizer_kwargs=minimizer_kwargs,
        take_step=None,
        accept_test=None,
        callback=None,
        interval=stepsize_interval,
        disp=False,
        niter_success=None,
        seed=None,
    )

In [ ]:
def printClassInitArgs(class_obj: bf.BenchmarkFunction) -> None:
    print(f"{class_obj.name()}")
    signature = inspect.signature(class_obj.__init__).parameters
    print("-------------------------------")
    for name, parameter in signature.items():
        if name != "opposite":
            print("Name: ", name, "\nDefault value:", parameter.default)
            # print("Annotation:", parameter.annotation, "\nKind:", parameter.kind)
            print("-------------------------------")

# Exercises

#### Solve the following exercises, and answer these questions at the end:

- How do the approaches seen in today's lab compare to the one seen in the previous lab?

In [ ]:
# BE AWARE: check the arguments each benchmark function takes
# if you're not sure, you can check the arguments by using the printClassInitArgs function
print(dir(bf))
printClassInitArgs(bf.Hypersphere())

## Exercise 1/2: DIRECT
In this first exercise we will use [DIRECT](https://scipydirect.readthedocs.io/en/latest/reference.html) as a search algorithm

More suited for functions with a single global optimum

### Questions
- How does the eps parameter influence the search process?
- How does the number of maximum iterations influence the search process? (To note: the maximum iterations and maximum number of evalutions are the soly metrics used to reach a stopping condition)

In [ ]:
benchmark = bf.Rosenbrock()

max_iter = 50
max_eval = 250

eps = [10.0, 1.0, 0.1, 0.01, 0.001]
for e in eps:
    func = OptFun(benchmark)
    res, best_location, best_value = direct(func, e, max_iter, max_eval)
    print("Function: ", func.name)
    print("Eps: ", e)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.DeJong3()

max_iter = 50
max_eval = 250

eps = [10.0, 1.0, 0.1, 0.01, 0.001]
for e in eps:
    func = OptFun(benchmark)
    res, best_location, best_value = direct(func, e, max_iter, max_eval)
    print("Function: ", func.name)
    print("Eps: ", e)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Keane()

max_iter = 50
max_eval = 250

eps = [10.0, 1.0, 0.1, 0.01, 0.001]
for e in eps:
    func = OptFun(benchmark)
    res, best_location, best_value = direct(func, e, max_iter, max_eval)
    print("Function: ", func.name)
    print("Eps: ", e)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Ackley()

max_iter = 50  # TODO: try different values
max_eval = 250  # TODO try different values

eps = [10.0, 1.0, 0.1, 0.01, 0.001]
for e in eps:
    func = OptFun(benchmark)
    res, best_location, best_value = direct(func, e, max_iter, max_eval)
    print("Function: ", func.name)
    print("Eps: ", e)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Rosenbrock()

max_iters = [50, 100, 500, 1000]
max_evals = [250, 500, 2500, 5000]

eps = 0.1
for max_iter in max_iters:
    for max_eval in max_evals:
        func = OptFun(benchmark)
        res, best_location, best_value = direct(func, eps, max_iter, max_eval)
        print("Function: ", func.name)
        print("Max iterations: ", max_iter)
        print("Max evaluations: ", max_eval)
        print("Minima value: ", func.minima()[0].score)
        print("Best value found:", best_value)
        print("Minima position: ", func.minima()[0].position)
        print("Best location found:", best_location)
        # print(res)
        func.plot()
        print("-------------------------------")

In [ ]:
benchmark = bf.DeJong5()

max_iters = [50, 100, 500, 1000]
max_evals = [250, 500, 1000, 2500]

eps = 0.01
for max_eval in max_evals:
    for max_iter in max_iters:
        func = OptFun(benchmark)
        res, best_location, best_value = direct(func, eps, max_iter, max_eval)
        print("Function: ", func.name)
        print("Max iterations: ", max_iter)
        print("Max evaluations: ", max_eval)
        print("Minima value: ", func.minima()[0].score)
        print("Best value found:", best_value)
        print("Minima position: ", func.minima()[0].position)
        print("Best location found:", best_location)
        print(res)
        func.plot()
        print("-------------------------------")

In [ ]:
benchmark = bf.DeJong5()

max_iters = [50, 100, 500, 1000]
max_evals = [250, 500, 1000, 2500]

eps = 0.01
for max_iter in max_iters:
    for max_eval in max_evals:
        func = OptFun(benchmark)
        res, best_location, best_value = direct(func, eps, max_iter, max_eval)
        print("Function: ", func.name)
        print("Max iterations: ", max_iter)
        print("Max evaluations: ", max_eval)
        print("Minima value: ", func.minima()[0].score)
        print("Best value found:", best_value)
        print("Minima position: ", func.minima()[0].position)
        print("Best location found:", best_location)
        print(res)
        func.plot()
        print("-------------------------------")

In [ ]:
# see the efforts in exploring the local minima
benchmark = bf.Griewank(zoom=1)

max_iter = 50  # TODO: try different values
max_eval = 250  # TODO try different values

eps = [0.001]
for e in eps:
    func = OptFun(benchmark)
    res, best_location, best_value = direct(func, e, max_iter, max_eval)
    print("Function: ", func.name)
    print("Eps: ", e)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    print(res)
    func.plot()

## Exercise 2/2: BASIN-HOPPING
In this exercise we will use [Basin-hopping](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html) as a search algorithm

More efffective on functions with multiple local minima

### Questions
- What is the influence of the following parameters on the search process?
  - **T:** The “temperature” parameter for the acceptance or rejection criterion. Higher “temperatures” mean that larger jumps in function value will be accepted. For best results T should be comparable to the separation (in function value) between local minima;
  - **stepsize:** Maximum step size for use in the random displacement;
  - **stepsize_interval:** interval for how often to update the stepsize;
  - **niter:** The number of basin-hopping iterations.
- How does the number of maximum iterations influence the search process?
- How does the starting point influence the search process?

In [ ]:
benchmark = bf.DeJong5()

x_0 = [0.0, 0.0]
stepsize = 10.0
stepsize_interval = 20
n_iter = 100

T = [100.0, 10.0, 1, 0.1, 0.01, 0.001]
for t in T:
    func = OptFun(benchmark)
    res = basinhopping(func, x_0, t, stepsize, stepsize_interval, n_iter, func.bounds())
    best_value, best_location = res["fun"], res["x"]

    print("Function: ", func.name)
    print("T: ", t)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.DeJong5()

x_0 = [10.0, 10.0]
stepsizes = [50.0, 10.0, 1.0, 0.1, 0.01]
stepsize_interval = 20
n_iter = 100

T = 0.1
for stepsize in stepsizes:
    func = OptFun(benchmark)
    res = basinhopping(func, x_0, T, stepsize, stepsize_interval, n_iter, func.bounds())
    best_value, best_location = res["fun"], res["x"]

    print("Function: ", func.name)
    print("T: ", T)
    print("Stepsize: ", stepsize)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Rastrigin()

x_0 = [-4.0, 4.0]
stepsize = 0.1
stepsize_interval = 25
n_iter = 100

T = [20.0]
for t in T:
    func = OptFun(benchmark)
    res = basinhopping(
        func, x_0, t, stepsize, stepsize_interval, n_iter, bounds=func.bounds()
    )
    best_value, best_location = res["fun"], res["x"]

    print("Function: ", func.name)
    print("T: ", t)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Keane()

x_0 = [8.0, 8.0]
stepsize = 1.5
stepsize_interval = 25
n_iter = 250

T = [100.0, 10.0, 1.0, 0.1, 0.01, 0.001]
for t in T:
    func = OptFun(benchmark)
    res = basinhopping(
        func, x_0, t, stepsize, stepsize_interval, n_iter, bounds=func.bounds()
    )
    best_value, best_location = res["fun"], res["x"]

    print("Function: ", func.name)
    print("T: ", t)
    print("Minima value: ", func.minima()[0].score)
    print("Best value found:", best_value)
    print("Minima position: ", func.minima()[0].position)
    print("Best location found:", best_location)
    # print(res)
    func.plot()

In [ ]:
benchmark = bf.Rosenbrock()

x_0 = [-2.0, -2.0]
n_iter = 25
T = 0.01

stepsize_interval = 10
stepsize = 0.1
func = OptFun(benchmark)
res = basinhopping(
    func, x_0, T, stepsize, stepsize_interval, n_iter, bounds=func.bounds()
)
best_value, best_location = res["fun"], res["x"]

print("Function: ", func.name)
print("Step size: ", stepsize)
print("Step size interval: ", stepsize_interval)
print("Minima value: ", func.minima()[0].score)
print("Best value found:", best_value)
print("Minima position: ", func.minima()[0].position)
print("Best location found:", best_location)
# print(res)
func.plot()

In [ ]:
benchmark = bf.EggHolder()

x_0 = [-2.0, -2.0]
n_iter = 100
T = 0.1

stepsize_intervals = [10, 25, 50, 100]
stepsizes = [1.0, 10.0, 50.0, 100.0]
for stepsize in stepsizes:
    for stepsize_interval in stepsize_intervals:
        func = OptFun(benchmark)
        res = basinhopping(
            func, x_0, T, stepsize, stepsize_interval, n_iter, func.bounds()
        )
        best_value, best_location = res["fun"], res["x"]

        print("Function: ", func.name)
        print("Step size: ", stepsize)
        print("Step size interval: ", stepsize_interval)
        print("Minima value: ", func.minima()[0].score)
        print("Best value found:", best_value)
        print("Minima position: ", func.minima()[0].position)
        print("Best location found:", best_location)
        # print(res)
        func.plot()

In [ ]:
benchmark = bf.DeJong3()

x_0 = [2.5, 2.5]
n_iter = 100
T = 1

stepsize_intervals = [10, 25, 50, 100]
stepsizes = [0.01, 0.1, 0.5, 1.0]
for stepsize in stepsizes:
    for stepsize_interval in stepsize_intervals:
        func = OptFun(benchmark)
        res = basinhopping(
            func, x_0, T, stepsize, stepsize_interval, n_iter, bounds=func.bounds()
        )
        best_value, best_location = res["fun"], res["x"]

        print("Function: ", func.name)
        print("Step size: ", stepsize)
        print("Step size interval: ", stepsize_interval)
        print("Minima value: ", func.minima()[0].score)
        print("Best value found:", best_value)
        print("Minima position: ", func.minima()[0].position)
        print("Best location found:", best_location)
        # print(res)
        func.plot()

## Final questions
- How do the approaches seen in today's lab compare to the one seen in the previous lab?

In [ ]:
# TODO